<a href="https://colab.research.google.com/github/sidgupta2205/TCAV_smai_team_9/blob/main/Experiment_activations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow/tcav.git tcav
%cd tcav
!ls

Cloning into 'tcav'...
remote: Enumerating objects: 700, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 700 (delta 122), reused 108 (delta 108), pack-reused 555
Receiving objects: 100% (700/700), 582.03 KiB | 10.78 MiB/s, done.
Resolving deltas: 100% (426/426), done.
/content/tcav
CONTRIBUTING.md        README.md	 Run_TCAV_on_colab.ipynb
FetchDataAndModels.sh  requirements.txt  setup.py
LICENSE		       Run_TCAV.ipynb	 tcav


In [ ]:
%cd /content/tcav/tcav/tcav_examples/image_models/imagenet
%run download_and_make_datasets.py --source_dir=YOUR_FOLDER --number_of_images_per_folder=10 --number_of_random_folders=10

/content/tcav/tcav/tcav_examples/image_models/imagenet
Created source directory at YOUR_FOLDER
INFO:tensorflow:Fetching imagenet data for zebra
INFO:tensorflow:Fetching imagenet data for zebra
INFO:tensorflow:Saving images at YOUR_FOLDER/zebra
INFO:tensorflow:Saving images at YOUR_FOLDER/zebra
INFO:tensorflow:Problem downloading imagenet image. Exception was HTTP Error 500: Internal Server Error for URL http://www.featurepics.com/FI/Marked/20060909/Zebra84929.jpg
INFO:tensorflow:Problem downloading imagenet image. Exception was HTTP Error 500: Internal Server Error for URL http://www.featurepics.com/FI/Marked/20060909/Zebra84929.jpg
INFO:tensorflow:Problem downloading imagenet image. Exception was HTTP Error 403: Forbidden for URL http://bonfire.learnnc.org/zoo/week02/photos/mccrary_zootales/images/zebra.jpg
INFO:tensorflow:Problem downloading imagenet image. Exception was HTTP Error 403: Forbidden for URL http://bonfire.learnnc.org/zoo/week02/photos/mccrary_zootales/images/zebra.jpg
I

In [ ]:
%cp -av '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224' '/content/tcav/tcav/mobilenet_v2_1.0_224'
%rm '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224'

'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224' -> '/content/tcav/tcav/mobilenet_v2_1.0_224'
'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224_eval.pbtxt' -> '/content/tcav/tcav/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224_eval.pbtxt'
'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224.ckpt.data-00000-of-00001' -> '/content/tcav/tcav/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224.ckpt.data-00000-of-00001'
'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224_frozen.pb' -> '/content/tcav/tcav/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224_frozen.pb'
'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224.tflite' -> '/content/tcav/tcav/mobilenet_v2_1.0_224/mobilenet_v2_1.0_224.tflite'
'/content/tcav/tcav/tcav_examples/image_models/imag

In [ ]:
%cp -av '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h' '/content/tcav/tcav/inception5h'
%rm '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h'

'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h' -> '/content/tcav/tcav/inception5h'
'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h/imagenet_comp_graph_label_strings.txt' -> '/content/tcav/tcav/inception5h/imagenet_comp_graph_label_strings.txt'
'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h/tensorflow_inception_graph.pb' -> '/content/tcav/tcav/inception5h/tensorflow_inception_graph.pb'
'/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h/LICENSE' -> '/content/tcav/tcav/inception5h/LICENSE'
rm: cannot remove '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h': Is a directory


In [ ]:
def import_graph(model_path, endpoints, image_value_range, scope='import'):
    t_input = tf.compat.v1.placeholder(np.float32, [None, None, None, 3])
    graph = tf.Graph()
    assert graph.unique_name(scope, False) == scope, (
        'Scope "%s" already exists. Provide explicit scope names when '
        'importing multiple instances of the model.') % scope

    graph_def = tf.compat.v1.GraphDef.FromString(
        tf.io.gfile.GFile(model_path, 'rb').read())

    with tf.compat.v1.name_scope(scope) as sc:
      t_input, t_prep_input = create_input(
          t_input, image_value_range)

      graph_inputs = {}
      graph_inputs[endpoints['input']] = t_prep_input
      myendpoints = tf.import_graph_def(
          graph_def, graph_inputs, list(endpoints.values()), name=sc)
      myendpoints = dict(list(zip(list(endpoints.keys()), myendpoints)))
      myendpoints['input'] = t_input
    return myendpoints

def get_bottleneck_tensors(scope):
    """Add Inception bottlenecks and their pre-Relu versions to endpoints dict."""
    graph = tf.compat.v1.get_default_graph()
    bn_endpoints = {}
    for op in graph.get_operations():
      if op.name.startswith(scope + '/') and 'Concat' in op.type:
        name = op.name.split('/')[1]
        bn_endpoints[name] = op.outputs[0]
    return bn_endpoints

def create_input(t_input, image_value_range):
    """Create input tensor."""
    def forget_xy(t):
      """Forget sizes of dimensions [1, 2] of a 4d tensor."""
      zero = tf.identity(0)
      return t[:, zero:, zero:, :]

    t_prep_input = t_input
    if len(t_prep_input.shape) == 3:
      t_prep_input = tf.expand_dims(t_prep_input, 0)
    t_prep_input = forget_xy(t_prep_input)
    lo, hi = image_value_range
    t_prep_input = lo + t_prep_input * (hi - lo)
    return t_input, t_prep_input

def _make_gradient_tensors(bottlenecks_tensors,loss):
    """Makes gradient tensors for all bottleneck tensors."""

    bottlenecks_gradients = {}
    for bn in bottlenecks_tensors:
      bottlenecks_gradients[bn] = tf.gradients(
          ys=loss, xs=bottlenecks_tensors[bn])[0]
    return bottlenecks_gradients


In [ ]:
def create_session(timeout=10000, interactive=True):
  """Create a tf session for the model.
  # This function is slight motification of code written by Alex Mordvintsev

  Args:
    timeout: tfutil param.

  Returns:
    TF session.
  """
  graph = tf.Graph()
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  config.operation_timeout_in_ms = int(timeout*1000)
  if interactive:
    return tf.compat.v1.InteractiveSession(graph=graph, config=config)
  else:
    return tf.compat.v1.Session(graph=graph, config=config)

def make_dir_if_not_exists(directory):
  if not tf.io.gfile.exists(directory):
    tf.io.gfile.makedirs(directory)

In [ ]:
from tensorflow.python.ops.gen_array_ops import shape
import os
import PIL.Image
import numpy as np

sess = None
bottlenecks_tensors = None
ends = None
examples = None
bottleneck_name = None

def run_examples(examples,bottleneck_name):
    global sess
    global bottlenecks_tensors
    global ends
    """Get activations at a bottleneck for provided examples.
    Args:
      examples: example data to feed into network.
      bottleneck_name: string, should be key of self.bottlenecks_tensors
    Returns:
      Activations in the given layer.
    """
    return sess.run(bottlenecks_tensors[bottleneck_name],
                         {ends['input']: examples})

def load_image_from_file(filename, shape):
  """Given a filename, try to open the file.
  If failed, return None.
  Args:
    filename: location of the image file
    shape: the shape of the image file to be scaled
  Returns:
    the image if succeeds, None if fails.
  Rasies:
    exception if the image was not the right shape.
  """
  normalize_image - True
  if not tf.io.gfile.exists(filename):
    tf.compat.v1.logging.error('Cannot find file: {}'.format(filename))
    return None
  try:
    # ensure image has no transparency channel
    img = np.array(
        PIL.Image.open(tf.io.gfile.GFile(filename,
                                          'rb')).convert('RGB').resize(
                                              shape, PIL.Image.BILINEAR),
        dtype=np.float32)
    if normalize_image:
      # Normalize pixel values to between 0 and 1.
      img = img / 255.0
    if not (len(img.shape) == 3 and img.shape[2] == 3):
      return None
    else:
      return img

  except Exception as e:
    tf.compat.v1.logging.info(e)
    return None
  return img

def load_images_from_files(filenames,
                            max_imgs=500,
                            do_shuffle=True,
                            run_parallel=True,
                            shape=(299, 299),
                            num_workers=50):
  """Return image arrays from filenames.
  Args:
    filenames: locations of image files.
    max_imgs: maximum number of images from filenames.
    do_shuffle: before getting max_imgs files, shuffle the names or not
    run_parallel: get images in parallel or not
    shape: desired shape of the image
    num_workers: number of workers in parallelization.
  Returns:
    image arrays
  """
  imgs = []
  # First shuffle a copy of the filenames.
  filenames = filenames[:]
  if do_shuffle:
    np.random.shuffle(filenames)
  
  for filename in filenames:
    img = load_image_from_file(filename, shape)
    if img is not None:
      imgs.append(img)
    if len(imgs) >= max_imgs:
      break
  if len(imgs) <= 1:
    raise ValueError(
        'You must have more than 1 image in each class to run TCAV.')

  return np.array(imgs)

def get_examples_for_concept(concept,shape):
    source_dir = '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER'
    concept_dir = os.path.join(source_dir, concept)
    img_paths = [
        os.path.join(concept_dir, d) for d in tf.io.gfile.listdir(concept_dir)
    ]
    imgs = load_images_from_files(
        img_paths, max_examples, shape)
    return imgs



def get_activations_for_examples(self, examples, bottleneck):
  acts = run_examples(examples, bottleneck)
  return np.asarray(acts).squeeze().squeeze()

def get_activations_for_concept(concept, bottleneck,shape):
    examples = get_examples_for_concept(concept,shape)
    return get_activations_for_examples(examples, bottleneck)

def process_and_load_activations(acts_dir,bottleneck_names, concepts,shape):
    acts = {}
    if acts_dir and not tf.io.gfile.exists(acts_dir):
      tf.io.gfile.makedirs(acts_dir)

    for concept in concepts:
      if concept not in acts:
        acts[concept] = {}
      for bottleneck_name in bottleneck_names:
        acts_path = os.path.join(acts_dir, 'acts_{}_{}'.format(
            concept, bottleneck_name)) if acts_dir else None
        if acts_path and tf.io.gfile.exists(acts_path):
          with tf.io.gfile.GFile(acts_path, 'rb') as f:
            acts[concept][bottleneck_name] = np.load(
                f, allow_pickle=True).squeeze()
            tf.compat.v1.logging.info('Loaded {} shape {}'.format(
                acts_path, acts[concept][bottleneck_name].shape))
        else:
          acts[concept][bottleneck_name] = get_activations_for_concept(
              concept, bottleneck_name,shape)
          if acts_path:
            tf.compat.v1.logging.info(
                '{} does not exist, Making one...'.format(acts_path))
            tf.io.gfile.mkdir(os.path.dirname(acts_path))
            with tf.io.gfile.GFile(acts_path, 'w') as f:
              np.save(f, acts[concept][bottleneck_name], allow_pickle=False)
    return acts

In [ ]:
from abc import abstractmethod
from six.moves import zip
import numpy as np
import six
import tensorflow as tf
from google.protobuf import text_format


model_to_run = 'GoogleNet'
# the name of the parent directory that results are stored (only if you want to cache)
project_name = 'tcav_class_test'
working_dir = '/content/tcav/tcav'
# where activations are stored (only if your act_gen_wrapper does so)
activation_dir =  working_dir+ '/activations/'
# where CAVs are stored. 
# You can say None if you don't wish to store any.
cav_dir = working_dir + '/cavs/'
# where the images live. 
source_dir = '/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER'
bottlenecks = [ 'mixed4c']  # @param 
      
make_dir_if_not_exists(activation_dir)
make_dir_if_not_exists(working_dir)
make_dir_if_not_exists(cav_dir)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]   

target = 'zebra'  
concepts = ["dotted","striped","zigzagged"] 
max_examples=500

#load model

model_path = "/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h/tensorflow_inception_graph.pb"
labels_path = "/content/tcav/tcav/tcav_examples/image_models/imagenet/YOUR_FOLDER/inception5h/imagenet_comp_graph_label_strings.txt"
labels = tf.io.gfile.GFile(labels_path).read().splitlines()
image_shape_v1 = [224, 224, 3]
image_value_range = (-117, 255 - 117)
endpoints_v1 = dict(
    input='input:0',
    logit='softmax2_pre_activation:0',
    prediction='output2:0',
    pre_avgpool='mixed5b:0',
    logit_weight='softmax2_w:0',
    logit_bias='softmax2_b:0',
)
scope='v1'
sess = create_session()
ends =  import_graph(model_path,endpoints_v1,image_value_range,scope)
bottlenecks_tensors = get_bottleneck_tensors(scope)
graph = tf.compat.v1.get_default_graph()
with graph.as_default():
  y_input = tf.compat.v1.placeholder(tf.int64, shape=[None])

  pred = tf.expand_dims(ends['prediction'][0], 0)
  loss = tf.reduce_mean(
      input_tensor=tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
          labels=tf.one_hot(
              y_input,
              ends['prediction'].get_shape().as_list()[1]),
          logits=pred))
bottleneck_gradients = _make_gradient_tensors(bottlenecks_tensors,loss)
model_name = 'GoogleNet_public'


normalize_image=True
acts = process_and_load_activations(activation_dir,bottlenecks, concepts,image_shape_v1[:2])
print(acts)
# graph = tf.compat.v1.get_default_graph()
# with graph.as_default():
#     y_input = tf.compat.v1.placeholder(tf.int64, shape=[None])
#     pred = tf.expand_dims(ends['prediction'][0], 0)
#     loss = tf.reduce_mean(
#         input_tensor=tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(
#             labels=tf.one_hot(
#                 y_input,
#                 ends['prediction'].get_shape().as_list()[1]),
#             logits=pred))
#     make_gradient_tensors()
# activations = sess.run(#layer name,images)
# activations x layer
# create dictionary of activations
# activations,labels,labels2txt
# intialise linear classifier
# num_correct += (sum(idx) * acc[labels2text[class_id]])
# acc['overall'] = float(num_correct) / float(len(y_test))
# self.cavs = [c for c in lm.coef_] linear classifier weights
# score = (target's example directional derivative -, count++)/total(examples)
# directional derivative = dot(cav,gradient of bootle neck wrt loss)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Loaded /content/tcav/tcav/activations/acts_dotted_mixed4c shape (10, 299, 299, 3)


INFO:tensorflow:Loaded /content/tcav/tcav/activations/acts_dotted_mixed4c shape (10, 299, 299, 3)


INFO:tensorflow:Loaded /content/tcav/tcav/activations/acts_striped_mixed4c shape (10, 299, 299, 3)


INFO:tensorflow:Loaded /content/tcav/tcav/activations/acts_striped_mixed4c shape (10, 299, 299, 3)


INFO:tensorflow:Loaded /content/tcav/tcav/activations/acts_zigzagged_mixed4c shape (10, 299, 299, 3)


INFO:tensorflow:Loaded /content/tcav/tcav/activations/acts_zigzagged_mixed4c shape (10, 299, 299, 3)


{'dotted': {'mixed4c': array([[[[0.6431373 , 0.6156863 , 0.5921569 ],
         [0.6431373 , 0.6156863 , 0.5921569 ],
         [0.6431373 , 0.6117647 , 0.5882353 ],
         ...,
         [0.57254905, 0.53333336, 0.49803922],
         [0.57254905, 0.53333336, 0.49803922],
         [0.57254905, 0.53333336, 0.49803922]],

        [[0.64705884, 0.61960787, 0.59607846],
         [0.6509804 , 0.61960787, 0.59607846],
         [0.64705884, 0.6156863 , 0.5921569 ],
         ...,
         [0.5764706 , 0.5372549 , 0.5019608 ],
         [0.5764706 , 0.5372549 , 0.5019608 ],
         [0.57254905, 0.53333336, 0.49803922]],

        [[0.654902  , 0.62352943, 0.6039216 ],
         [0.65882355, 0.62352943, 0.6       ],
         [0.654902  , 0.61960787, 0.59607846],
         ...,
         [0.5803922 , 0.5411765 , 0.5058824 ],
         [0.5764706 , 0.5372549 , 0.5019608 ],
         [0.57254905, 0.53333336, 0.49803922]],

        ...,

        [[0.5372549 , 0.50980395, 0.47843137],
         [0.5254902 , 